# Homework — GEIM and Data Assimilation (v2)

## Learning objectives

This homework explores the **Generalized Empirical Interpolation Method (GEIM)** for:
- Building sensor dictionaries with different linear functionals
- Greedy basis construction with simultaneous sensor selection
- Data assimilation from sparse measurements
- Stability analysis (Lebesgue constants, conditioning)
- Robustness to noise via overdetermination and regularization

**Key concepts:** Linear functionals, greedy algorithms, interpolation theory, least squares, Tikhonov regularization.

**Tools:** NumPy for numerics, Matplotlib for visualization.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
np.random.seed(7)

## 1. Problem setup

In [2]:
Nq = 800
x = np.linspace(0.0, 1.0, Nq)
mu_box = np.array([[0.0,2.0],[0.0,2.0],[0.2,0.8]])
def s_fun(x, mu):
    mu1,mu2,mu3 = mu
    return (1+mu1)*np.sin((1+mu2)*np.pi*x) + np.exp(-20.0*(x-mu3)**2)
def sample_mu(n):
    u = np.random.rand(n,3)
    return mu_box[:,0] + u*(mu_box[:,1]-mu_box[:,0])
n_train, n_test = 300, 150
mu_train, mu_test = sample_mu(n_train), sample_mu(n_test)
assert mu_train.shape==(n_train,3) and mu_test.shape==(n_test,3)

## 2. Sensor dictionary $A$
TODO: implement Dirac-like probes (subsampled), box averages, Gaussian sensors; normalize rows.

In [ ]:
def make_dictionary(x, stride=10, n_boxes=16, box_width=0.06, n_gauss=16, sigma=0.035):
    # --- TODO ---
    raise NotImplementedError('make_dictionary')

### Utility: forward substitution (unit diagonal)

In [ ]:
def forward_solve(L, b):
    # --- TODO ---
    raise NotImplementedError('forward_solve')

## 3. GEIM offline (greedy)

**GEIM idea.** Build basis functions $\{\rho_j\}$ and a set of **sensors** $\{L_i\}$ such that
$$ L_i\big(I_M g\big) = L_i(g), \quad i=1,\dots,M, $$
with $I_M g(x) = \sum_{j=1}^M \gamma_j(\mu)\,\rho_j(x)$ and the interpolation matrix $B_M = [L_i(\rho_j)]$ **lower‑triangular with ones on the diagonal** (by normalization).

**Task B.** Implement the GEIM greedy algorithm:
1. Precompute training snapshots `S` on the grid for `mu_train`.
2. **Select** $\mu_1$ maximizing $\|g(\cdot;\mu)\|_\infty$; set $\xi_1=g(\cdot;\mu_1)$. **Choose** $L_1$ maximizing $|L(\xi_1)|$ over the dictionary and set $\rho_1=\xi_1 / L_1(\xi_1)$.
3. For $m=1,\dots,M-1$:
   - Pick $\mu_{m+1}$ maximizing the current interpolation error $\|g-I_m g\|_\infty$ over the training set.
   - Compute the residual $r_{m+1}=\xi_{m+1}-I_m\xi_{m+1}$.
   - **Choose** $L_{m+1}$ in the dictionary (not yet used) maximizing $|L(r_{m+1})|$.
   - Set $\rho_{m+1}=r_{m+1}/L_{m+1}(r_{m+1})$.

Store: `Q` ($N_q\times M$), sensor indices list `J` (rows of `A`), selected training parameters, training errors, and (optionally) a discrete Lebesgue constant.

*Hints.*
- To reconstruct a snapshot with current $(Q_m, J_m)$: build $B_m=A[J_m,:]@Q_m$, measure $g_I=A[J_m,:]@g$, solve $\gamma=\text{forward\_solve}(B_m, g_I)$, then `approx = Q_m @ gamma`.
- With the normalization above, `B_m` is lower‑triangular with a unit diagonal by construction.


In [ ]:
@dataclass
class GEIMModel:
    Q: np.ndarray
    J: list
    mu_sel: list
    train_err: list
    lebesgue: list

def geim_offline(s_fun, x, mu_train, A, M_max=16, tol=1e-6):
    # --- TODO ---
    raise NotImplementedError('geim_offline')

## 4. GEIM online (assimilation)


**Task C.** Given `model` and a set of sensor measurements `y` at the selected sensors (rows `J`), reconstruct the field $g_M$ by solving
$$ B_M\,\gamma = y, \quad B_M = A[J,:]@Q, \quad g_M = Q\,\gamma. $$
Return `(gM, gamma)`.

In [ ]:
def geim_online(model, A, y):
    # --- TODO ---
    raise NotImplementedError('geim_online')

## 5. Sanity checks (after offline/online)


Check triangularity/unit diagonal of B=A[J,:]@Q and interpolation at selected sensors. 

After you implement `make_dictionary` and `geim_offline`, build a model with `M_max≈20` and verify:
- `B = A[J,:] @ Q` is lower‑triangular with (approximately) unit diagonal.
- Interpolation holds at selected sensors on the training snapshots: `A[J,:] @ (Q @ gamma) == A[J,:] @ g` for all training `g` when `gamma` solves `B gamma = A[J,:] @ g`.

In [5]:
# A = make_dictionary(x)
# model = geim_offline(s_fun, x, mu_train, A, M_max=16, tol=1e-8)
# B = A[model.J,:] @ model.Q
# assert np.allclose(np.tril(B), B)
# assert np.allclose(np.diag(B), 1.0, atol=1e-12)
# S_train = np.stack([s_fun(x, mu) for mu in mu_train], axis=1)
# rhs = A[model.J,:] @ S_train
# gamma = forward_solve(B, rhs)
# interp = model.Q @ gamma
# assert np.allclose(A[model.J,:] @ interp, rhs, atol=1e-10)

## 6. Data assimilation: noise-free

**Task D.** Pick a fresh parameter `mu0`, generate measurements `y = A[J,:] @ s_fun(x, mu0)`, reconstruct with `geim_online`, and report the sup‑norm error `||truth - gM||_∞`. Plot the truth vs reconstruction.


In [ ]:
# mu0 = sample_mu(1)[0]
# truth = s_fun(x, mu0)
# y = (A[model.J,:] @ truth)
# gM, _ = geim_online(model, A, y)
# print('sup error (noise-free):', float(np.max(np.abs(truth-gM))))
# plt.figure(figsize=(6,4))
# plt.plot(x, truth, label='truth')
# plt.plot(x, gM, '--', label='GEIM')
# plt.legend()
# plt.grid(True)
# plt.show()

## 7. Data assimilation: noisy and gappy (optional)

Add i.i.d. Gaussian noise of level `sigma_y` to the measurements: `y^δ = y + δ`. Run multiple trials and compute the mean and standard deviation of the reconstruction error.

**Task E (deterministic GEIM).** Repeat Task D with noisy measurements (same $M$). Study sensitivity as a function of (i) $M$, and (ii) the conditioning of $B_M$.

**Task F (gappy‑GEIM: overdetermined).** Use **more sensors than basis functions** (|I| > M). Solve the least‑squares problem
$$\min_\gamma \; \|A[I,:]Q\,\gamma - y^\delta\|_2^2 + \tau\,\|\gamma\|_2^2,$$
for some Tikhonov parameter $\tau \ge 0$. Compare errors vs the square case. *Hint:* use the normal equations or `np.linalg.lstsq` when $\tau=0$; with Tikhonov, solve $(B^TB+\tau I)\gamma=B^Ty$ with $B=A[I,:]Q$.

In [ ]:
def geim_online_gappy(Q, A, I_rows, y, tau=0.0):
    # --- OPTIONAL TODO ---
    raise NotImplementedError('geim_online_gappy')

## 8. Diagnostics
TODO: test-set sup error vs M; discrete Lebesgue constants; cond(B_m). **Task G.**
1) **Test‑set error curve**: for each $m=1,\dots,M$, reconstruct all test functions from measurements at the first $m$ sensors, and report $\max_{x,\mu\in\text{test}}|s-I_m s|$.
2) **Discrete Lebesgue constant** (GEIM): compute $\Lambda_m = \max_x \sum_i |(Q_m B_m^{-1})[x,i]|$ where $B_m=A[J_m,:]Q_m$. Plot growth of $\Lambda_m$.
3) **Conditioning**: plot $\kappa(B_m)$ vs $m$. Relate error sensitivity to $\Lambda_m$ and $\kappa(B_m)$.

*Hints:* These are identical to the EIM algebra with $Q$ and $B$ redefined using functionals. Compare with the lecture’s EIM error analysis and a posteriori one‑point result (you can define a “one‑functional estimator” by evaluating at the next chosen functional).

In [6]:
# def test_error_curve_geim(model, x, mu_test, A):
#     # --- TODO ---
#     raise NotImplementedError()
#
# def discrete_lebesgue_constants_geim(Q, J, A):
#     # --- TODO ---
#     raise NotImplementedError()
#
# def cond_numbers_geim(Q, J, A):
#     # --- TODO ---
#     raise NotImplementedError()